# SDP system sizing

This notebook generates the base numbers that go into SDP system sizing. We use the list of high priority science objectives (HPSO) to derive the average SDP computational load and output data rate. 

In [ ]:
import sys
from IPython.display import display, Markdown

sys.path.insert(0, "..")
from sdp_par_model import reports, config
from sdp_par_model.parameters.definitions import Telescopes, Pipelines, Constants, HPSOs

## Read HPSO performance characteristics

Loads high performance science objective characteristics generated by the export notebook. This always picks up the latest file checked into Git.

In [ ]:
csv = reports.read_csv(reports.newest_csv(reports.find_csvs()))
csv = reports.strip_csv(csv)
def lookup(name, hpso):
    return { pipeline : float(reports.lookup_csv(csv, config.PipelineConfig(hpso=hpso, pipeline=pipeline).describe(), name))
             for pipeline in HPSOs.hpso_pipelines[hpso] }
total_time = { tel:
    sum([lookup('Total Time', hpso).get(Pipelines.Ingest)
        for hpso in HPSOs.all_hpsos if HPSOs.hpso_telescopes[hpso] == tel])
    for tel in Telescopes.available_teles
}

## Compute requirements

Sum up computational requirements of HPSOs, weighted by allocated fraction of observation time. Note that HPSOs do not cover the whole breath of what SKA SDP will be required to do, but we assume that the load is representative.

In [ ]:
def make_compute_table(tel):
    table = [ "| HPSO | Time [%] | Tobs [h] | Ingest [Pflop/s] | RCAL [Pflop/s] | FastImg [Pflop/s] | ICAL [Pflop/s] " +
                "| DPrepA [Pflop/s] | DPrepB [Pflop/s] | DPrepC [Pflop/s] | DPrepD [Pflop/s] " +
                "| Total RT [Pflop/s] | Total Batch [Pflop/s] | Total [Pflop/s] | "]
    table.append("-".join("|"*table[0].count('|')))
    flop_sum = { pip : 0 for pip in Pipelines.available_pipelines }
    pips = [ Pipelines.Ingest, Pipelines.RCAL, Pipelines.FastImg,
             Pipelines.ICAL, Pipelines.DPrepA, Pipelines.DPrepB, Pipelines.DPrepC, Pipelines.DPrepD,]
    for hpso in sorted(HPSOs.all_hpsos):
        if HPSOs.hpso_telescopes[hpso] != tel:
            continue
        Tobs = lookup('Observation Time', hpso).get(Pipelines.Ingest,0)
        Texp = lookup('Total Time', hpso).get(Pipelines.Ingest,0)
        flops = lookup('Total Compute Requirement', hpso)
        Rflop = sum(flops.values())
        Rflop_rt = sum([ Rflop for pip, Rflop in flops.items() if pip in Pipelines.realtime])
        time_frac = Texp / total_time[tel]
        for pip, rate in flops.items():
            flop_sum[pip] += time_frac * rate
        flops_strs = ["{:.2f}".format(flops[pip]) if pip in flops else '-' for pip in pips]
        table.append("|{}|{:.1f}|{:.1f}|{}|{}|{}|{}|{}|{}|{}|{}|{:.2f}|{:.2f}|{:.2f}|".format(
            hpso,time_frac*100,Tobs/3600,*flops_strs,Rflop_rt,Rflop-Rflop_rt,Rflop))
    table.append("| **Average** | - | - | {:.2f}|{:.2f}|{:.2f}|{:.2f}|{:.2f}|{:.2f}|{:.2f}|{:.2f}|{:.2f}|{:.2f}|{:.2f}|".format(
        *[flop_sum.get(pip,0) for pip in pips],
        sum([ Rflop for pip, Rflop in flop_sum.items() if pip in Pipelines.realtime]),
        sum([ Rflop for pip, Rflop in flop_sum.items() if pip not in Pipelines.realtime]),
        sum(flop_sum.values())))
    return "\n".join(table)
for tel in Telescopes.available_teles:
    display(Markdown("##### {}:\n\n".format(tel) + make_compute_table(tel)))

Note that this is minimum requirements, in reality we will need substantially more capacity to compensate for unavoidable inefficiencies in scheduling. See the scheduling notebook for a model of this.

## Data Product generation

Similar analysis, but for the data rate at which SDP  might generate data products.

In [ ]:
def make_data_table(tel, return_average=False):
    table = [ "| HPSO | Time [%] | Tobs [h] | Npix (side) | Channels (DPrepB) | Channels (DPrepC) | " +
              " Image size [GB] | Non-Vis Rate [Gbit/s] | " +
              " Visibility Size [TB] | Visibility Rate [Gbit/s] | Total Rate [Gbit/s] | "]
    table.append("-".join("|"*table[0].count('|')))    
    vis_sum = 0; output_sum = 0
    for hpso in sorted(HPSOs.all_hpsos):
        if HPSOs.hpso_telescopes[hpso] != tel:
            continue
        Tobs = lookup('Observation Time', hpso).get(Pipelines.Ingest,0)
        Texp = lookup('Total Time', hpso).get(Pipelines.Ingest,0)
        time_frac = Texp / total_time[tel]
        Mout = sum(lookup('Output size', hpso).values())
        Mvis = lookup('Output size', hpso).get(Pipelines.DPrepD,0)
        Rout = 8000 * Mout / Tobs; Rvis = 8000 * Mvis / Tobs
        vis_sum += Rvis * time_frac; output_sum += Rout * time_frac
        row = "|{}|{:.1f}|{:.2f}|".format(hpso,time_frac*100,Tobs/3600,Mout,Rvis,Rout)
        if Pipelines.DPrepA in HPSOs.hpso_pipelines[hpso]:
            Npix = lookup('Image side length', hpso).get(Pipelines.DPrepA,0)
            Nchan_B = lookup('Channels out', hpso).get(Pipelines.DPrepB,0)
            Nchan_C = lookup('Channels out', hpso).get(Pipelines.DPrepC,0)
            Mimage = lookup('Image size', hpso).get(Pipelines.DPrepA,0)
            row += "{:.0f}|{:.0f}|{:.0f}|{:.1f}|{:.1f}|".format(
                Npix,Nchan_B,Nchan_C,Mimage,Rout-Rvis)
        else:
            row += "- | - | - | - |{:.1f}|".format(Rout-Rvis)
        if Pipelines.DPrepD in HPSOs.hpso_pipelines[hpso]:
            row += "{:.1f}|{:.1f}|".format(Mvis, Rvis)
        else:
            row += "-|-|"
        row += "{:.1f}|".format(Rout)
        table.append(row)
    table.append("| **Average** | - | - | - | - | - | - | {:.1f} | - |{:.1f}|{:.1f}|".format(
        output_sum-vis_sum, vis_sum, output_sum))
    if return_average:
        return output_sum
    else:
        return "\n".join(table)
for tel in Telescopes.available_teles:
    display(Markdown("##### {}:\n\n".format(tel) + make_data_table(tel)))

Note that this assumes that we manage to produce usable data at all times.

We can make a similar graph where we sum the weighted contributions of all HPSOs per pipeline:

In [ ]:
table = [ "| Telescope | Pipeline | Data Rate [Gbit/s] | Daily Growth [TB/day] | Yearly Growth [PB/year] | 5-year Growth [PB/(5 year)] |"]
table.append("-".join("|"*table[0].count('|')))
total_data_rate = 0
for tel in Telescopes.available_teles:
    def mk_projection(rate):
        day_rate = rate * 3600 * 24 / 8 / 1000 # TB/day
        year_rate = day_rate * 365 / 1000 # PB/year
        return "{:.2f}|{:.1f}|{:.1f}|{:.1f}".format(rate, day_rate, year_rate, 5*year_rate)
    subtotal_data_rate = 0
    for pip in Pipelines.all:
        data_rate = 0
        for hpso in HPSOs.all_hpsos:
            if HPSOs.hpso_telescopes[hpso] == tel and pip in HPSOs.hpso_pipelines[hpso]:
                Texp = lookup('Total Time', hpso).get(Pipelines.Ingest,0)
                Tobs = lookup('Observation Time', hpso).get(Pipelines.Ingest,0)
                Mout = lookup('Output size', hpso).get(pip)
                Rout = 8000 * Mout / Tobs
                time_frac = Texp / total_time[tel]
                data_rate += Rout * time_frac
        if data_rate > 0:
            table.append("|{}|{}|{}|".format(tel, pip, mk_projection(data_rate)))
            subtotal_data_rate += data_rate; total_data_rate += data_rate
    table.append("|**{}**|**Sub-Total**|{}|".format(tel, mk_projection(subtotal_data_rate)))
table.append("|&nbsp;|**Total**|{}|".format(mk_projection(total_data_rate)))
display(Markdown("\n".join(table)))

This gives a good idea of how the archive will need to grow, and what types of data products it will contain. For instance, DPrepD will produce visibility data products (mostly for EoR search) that might eventually be discarded after delivery.